# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending a question to it and getting it scored for matches in the original questions.

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app
- Tear it all down

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

## Setup

Below are the various name definitions for the resources needed to setup AKS as well as the name of the Docker image we will be using.

In [1]:
# Please modify the below as you see fit
resource_group = "<RESOURCE_GROUP>" 
aks_name = "<AKS_CLUSTER_NAME>"
location = "eastus"

docker_login = '<YOUR_DOCKER_LOGIN>'
image_name = docker_login + '/mlaksdep' # 'fboylu/mlaksdep' Use this image if you want to skip creating your own container
selected_subscription = "'<YOUR_SUBSCRIPTION>'" # If you have multiple subscriptions select the subscription you want to use 

In [2]:
resource_group = "fbakscpurg" # Feel free to modify these
aks_name = "fbAKSClustercpu"
location = "eastus"

docker_login = 'fboylu'
image_name = docker_login + '/mlaksdep' 
selected_subscription = "'Team Danielle Internal'"

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [5]:
!az login -o table

In [7]:
!az account set --subscription $selected_subscription

In [6]:
!az account show

You will also need to register the container service resources on your subscription if you haven't already done so.

In [7]:
!az provider register -n Microsoft.ContainerService

In [8]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [3]:
!az group create --name $resource_group --location $location

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/fbaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "fbaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster  with 5 nodes in the resource group we created earlier.

In [6]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 5 --generate-ssh-keys -s Standard_D4_v2

{- Finished ..
  "agentPoolProfiles": [
    {
      "count": 1,
      "dnsPrefix": null,
      "fqdn": null,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "ports": null,
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "fbAKSClust-fbaksrg-edf507",
  "fqdn": "fbaksclust-fbaksrg-edf507-e12d7f40.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/fbaksrg/providers/Microsoft.ContainerService/managedClusters/fbAKSClustergpu",
  "kubernetesVersion": "1.9.6",
  "linuxProfile": {
    "adminUsername": "azureuser",
    "ssh": {
      "publicKeys": [
        {
          "keyData": "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDTZYQFHNstYCR25qtvMrC6baTMS6TobaIRbgd0xOoafDy+2uBk0DMJuhGWoOcrsCnvadp5k/0K8qBRysyhlQGWb6+r8fBunThy+zpTKqdh3W8Q1y5UtKnGwwU1cqGXDOPUIXJYNPJqUKV829+MOrZjUynhHgSzDbY2ncGyoT+Farsvm01aGEdDapa+XRl4JAwtN1bb9q+Ii5y+MkpIOhLRMwATl05

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [9]:
!sudo env "PATH=$PATH" az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [10]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSClustercpu" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [11]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-13347348-0   Ready     agent     40d       v1.9.6
aks-nodepool1-13347348-1   Ready     agent     40d       v1.9.6
aks-nodepool1-13347348-2   Ready     agent     40d       v1.9.6
aks-nodepool1-13347348-3   Ready     agent     40d       v1.9.6
aks-nodepool1-13347348-4   Ready     agent     40d       v1.9.6


Let's check the pods on our cluster.

In [17]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          40d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          40d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          40d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          40d
kube-system   kube-proxy-225hz                        1/1       Running   0          40d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          40d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          40d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          40d
kube-system   kube-proxy-tvnpv                        1/1       Running   0          40d
kube-system   kube-svc-redirect-278jk                 1/1       Running   3          40d
kube-system   kube-sv

## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the image name and cpu requests and limits for pods. We first start with  deploying 2 pods.

In [20]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "replicas":2,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-ml"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-ml",
                      "image": image_name,

                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "cpu": 1
                           },
                           "limits":{
                               "cpu": 1.25
                           }
                       }  
                  }
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-ml"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-ml"
      }
   }
}

In [21]:
import json
def write_json_to_file(json_dict, filename, mode='w'):
    with open(filename, mode) as outfile:
        json.dump(json_dict, outfile, indent=4, sort_keys=True)
        outfile.write('\n\n')

In [24]:
write_json_to_file(app_template, 'az-ml.json')

In [25]:
write_json_to_file(service_temp, 'az-ml.json', mode='a')

Let's check the manifest created.

In [27]:
!cat az-ml.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-ml"
    },
    "spec": {
        "replicas": 2,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-ml"
                }
            },
            "spec": {
                "containers": [
                    {
                        "image": "fboylu/mlaksdep",
                        "name": "azure-ml",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                        "resources": {
                            "limits": {
                                "cpu": 1.25
                            },
                            "requests": {
                                "cpu": 1
                            }
                        }
              

Next, we will use kubectl create command to deploy our application.

In [28]:
!kubectl create -f az-ml.json

deployment.apps/azure-ml created
service/azure-ml created


Let's check if the pod is deployed.

In [44]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-ml-7d9b465b7f-dwg6p               1/1       Running   0          4m
default       azure-ml-7d9b465b7f-lbzz4               1/1       Running   0          4m
kube-system   azureproxy-79c5db744-j5hj4              1/1       Running   3          40d
kube-system   heapster-55f855b47-bs86x                2/2       Running   0          40d
kube-system   kube-dns-v20-7c556f89c5-96dfd           3/3       Running   0          40d
kube-system   kube-dns-v20-7c556f89c5-fm7v2           3/3       Running   0          40d
kube-system   kube-proxy-225hz                        1/1       Running   0          40d
kube-system   kube-proxy-g5x5p                        1/1       Running   0          40d
kube-system   kube-proxy-hw7fk                        1/1       Running   0          40d
kube-system   kube-proxy-ttnsh                        1/1       Running   0          40d
kube-system   kube-prox

If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [45]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                       SOURCE                              MESSAGE
4m          4m           1         azure-ml-7d9b465b7f-dwg6p.1542cd4b42ad11a0   Pod                                      Normal    Scheduled                    default-scheduler                   Successfully assigned azure-ml-7d9b465b7f-dwg6p to aks-nodepool1-13347348-4
4m          4m           1         azure-ml-7d9b465b7f-dwg6p.1542cd4b525e6208   Pod                                      Normal    SuccessfulMountVolume        kubelet, aks-nodepool1-13347348-4   MountVolume.SetUp succeeded for volume "default-token-lls54" 
2m          4m           2         azure-ml-7d9b465b7f-dwg6p.1542cd4b846d7ba5   Pod          spec.containers{azure-ml}   Normal    Pulling                      kubelet, aks-nodepool1-13347348-4   pulling image "fboylu/mlaksdep"
2m          2m           1         azu

Check the logs for the application pod.

In [46]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [47]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-07-19 15:06:16,449 CRIT Supervisor running as root (no user in config file)
2018-07-19 15:06:16,452 INFO supervisord started with pid 1
2018-07-19 15:06:17,454 INFO spawned: 'program_exit' with pid 11
2018-07-19 15:06:17,456 INFO spawned: 'nginx' with pid 12
2018-07-19 15:06:17,458 INFO spawned: 'gunicorn' with pid 13
2018-07-19 15:06:18,568 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
{"message": "Model object loading time: 523.19 ms", "stack_info": null, "timestamp": "2018-07-19T15:06:19.069806Z", "host": "azure-ml-7d9b465b7f-dwg6p", "tags": [], "path": "/code/driver.py", "logger": "model_driver", "level": "INFO"}
Initialising
{"msg": " * Running on %s://%s:%d/ %s", "message": " * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)", "stack_info": null, "timestamp": "2018-07-19T15:06:19.143402Z", "host": "azure-ml-7d9b465b7f-dwg6p", "tags": [], "path": "/opt/conda/envs/py3.5/lib/python3.5/site-packages/werkzeug/_in

In [48]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-ml   2         2         2            2           4m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [50]:
!kubectl get service azure-ml

NAME       TYPE           CLUSTER-IP    EXTERNAL-IP      PORT(S)        AGE
azure-ml   LoadBalancer   10.0.33.114   137.117.85.251   80:31301/TCP   5m


# Scaling

In this part, we provide commands to scale the number of pods.

In [52]:
!kubectl scale --current-replicas=2 --replicas=35 deployment/azure-ml

deployment.extensions/azure-ml scaled


In [55]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-ml-7d9b465b7f-245fv               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-274qk               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-4h5pd               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-5qwlz               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-66lcp               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-72dww               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-7cx6l               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-8hzwc               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-dwg6p               1/1       Running   0          1h
default       azure-ml-7d9b465b7f-dwlhg               1/1       Running   0          1h
default       azure-ml-7d9b465b

In [56]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-ml   35        35        35           35          1h


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb). Once, we are done with all the notebooks of the tutorial, below instructions can be used to delete the cluster and free resources

# Tear it all down

Once you are done with your cluster you can use the following two commands to destroy it all.

In [48]:
!kubectl delete -f az-ml.json

deployment.apps "azure-dl" deleted
service "azure-dl" deleted


In [3]:
!az aks delete -n $aks_name -g $resource_group -y

In [4]:
!az group delete --name $resource_group -y

 - Finished ..